In [ ]:
SKIP_STEPS = 10
LOWER_RES = 60

In [ ]:
import csv
import numpy as np
import pandas
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy.random import normal as sample1d
import cv2
from PIL import Image 
import numpy as np
from math import sin, cos, pi
from numpy.random import uniform

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_randrad(pos, radius, num):
    initial_set = []
    for i in range(num):
      radius = np.random.uniform() * radius
      angle = np.random.uniform() * 2 * pi
      initial_set.append([pos[0] + radius * cos(angle), pos[1] + radius * sin(angle)])
    initial_set = np.array(initial_set)
    return initial_set

In [ ]:
def initiate_particles(intial_state, num):
    paticles = np.ones((num,3))
    noisy_xy = get_randrad(np.array([intial_state[0], intial_state[1]]), 300, num)
    paticles[:, 0] *= noisy_xy[:,0]
    paticles[:, 1] *= noisy_xy[:,1]
    for i in range(num):
      paticles[i, 2] *= intial_state[2] + np.random.uniform() * 0.1 * pi
      
    return paticles    

In [ ]:
def propagate_particle(state, action, cov, alphas): 
    ''' state: the particle old state np.array() (3,) [x,y,theat]
    action: control action np.array() 2,, [dtran, drot]
    noise_cov: the motion model noise covariance. for this paper the estimated error standard deviation is 1/3
    alphas:  measurement noise scale coefficients, selected manually, np.array() (3,) [alpha1, alpha2, alpha3]
    ''' 
    state_new = np.ones_like(state)
    dtran = action[0] + sample1d(0, cov)
    drot = action[1] + sample1d(0, cov)
    state_new[0] = state[0]+alphas[0]*dtran*np.cos(state[2]+alphas[2]*drot)
    state_new[1] = state[1]+alphas[1]*dtran*np.sin(state[2]+alphas[2]*drot)
    state_new[2] = wrap_angle(state[2]+alphas[2]*drot)
    return state_new

In [ ]:
'''
I - image from camera
T - image from map from particle
They should have same dimensions
Written with numpy functions 
'''
def logistic(x, v):
    return (1/((1+np.exp(-5*x))**(1/v)))

def find_r(I, T, v):
    W = I.shape[0]
    H = I.shape[1]
    T_ = T - np.sum(T)/(W*H)
    I_ = I - np.sum(I)/(W*H)
    r = np.sum(T_*I_)/((np.sum(I_**2)**0.5)*(np.sum(T_**2)**0.5))
    #print('R = ', r)
    #logistic transform
    L_r = logistic(r,v)/logistic(1,v)
    return L_r

In [ ]:
'''
 Map - whole map
 pos - np.array([x, y, theta]) from particles
 dim - dimension of camera picture [Width, Height]
'''
def rotate_pic(Map, pos, dim):
    W = dim[0]
    H = dim[1]
    
    theta = pos[2]
    Tr_Matrix = np.array([[cos(theta), sin(theta)],[-sin(theta), cos(theta)]])

    Old_Indexes = []
    X = np.array(range(-W//2+1, W//2+1))
    Y = np.array(range(-H//2+1, H//2+1))
    for i in X:
        for j in Y:
            Old_Indexes.append([i, j])
            
    Old_Indexes = np.array(Old_Indexes).T
    New_Indexes = np.array(Tr_Matrix@Old_Indexes + pos[:2].reshape(2,1), dtype = np.int16 )

    Cut_im = np.ones([W,H])
    n = 0

    for x in range(W):
        for y in range(H):
            Cut_im[x,y] = Map[New_Indexes[0,n],New_Indexes[1,n]]
            n += 1
    plt.title('cropped Image')
    plt.imshow(Cut_im, cmap='gray',origin='lower')
    plt.show()
    return Cut_im

In [ ]:
def get_prob(Imag, Map, Particles, v):
    Samples = []
    W = int(Imag.shape[0])
    H = int(Imag.shape[1])

    for i in range(Particles.shape[0]):
        pos = Particles[i]       #[x, y, theta]
        T = rotate_pic(Map, pos, [W,H])
        F_r = find_r(Imag, T, v)
        Samples.append(F_r)
    
    Samples = np.array(Samples)
    return F_r

In [ ]:
def get_svo_commands(svo_x, svo_y):
    current_position = np.array([0,0,0])
    drot = []
    dtrans = []

    for i in range(1, svo_x.shape[0], SKIP_STEPS):
        x_position = svo_x[i]
        y_position = svo_y[i]
        # Deduce Motion commands
        drot_temp = np.arctan2(y_position - current_position[1], x_position - current_position[0]) - current_position[2]
        dtrans_temp = np.sqrt((x_position - current_position[0])**2 + (y_position - current_position[1])**2)
        # update current position
        current_position = np.array([x_position, y_position, current_position[2] + drot_temp])
        drot.append(drot_temp)
        dtrans.append(dtrans_temp)

    dtrans = np.array(dtrans)
    drot = np.array(drot)
    return dtrans, drot

In [ ]:
'''
Wraps the given angle to the range [-pi, +pi].
param angle: The angle (in rad) to wrap (can be unbounded).
return: The wrapped angle (guaranteed to in [-pi, +pi]).'''

def wrap_angle(angle):
    pi2 = 2 * np.pi
    while angle < -np.pi:
        angle += pi2
    while angle >= np.pi:
        angle -= pi2
    return angle

In [ ]:
import math
 
def euler_from_quaternion(x, y, z, w):
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return roll_x, pitch_y, yaw_z 

In [ ]:
def get_grid_size(map_cor, gt_cor, cor='x'):
  grid_size = np.arange(0.5, 1.7, 0.01)
  error = np.inf
  for i in grid_size:
    if cor == 'x':
      gt_px = (gt_cor//i) + map_cor[0]
    else:
      gt_px = (-gt_cor//i) + map_cor[0]
    error_i = np.sum(np.abs(gt_px - map_cor))
    if error_i <= error:
      error = error_i
      result = i
  return result

In [ ]:
def reducing_res_img(Image, div):
    New = np.zeros([int(Image.shape[0]/div), int(Image.shape[1]/div)])
    for i in range(New.shape[0]):
        for j in range(New.shape[1]):
            New[i,j] = Image[int(i*div),int(j*div)]
    
    return New

In [ ]:
def unit_low_variance_sampling(states, weights, i, c, r, m):
   M = states.shape[0]
   U = r + (m)*(1/M)
   while U > c: 
     i = i + 1
     c = c + weights[i]
     if i == M-1:
       i = 0
   resampled_p = states[i]
   resampled_w = weights[i]
   return resampled_p, resampled_w, i, c  

In [ ]:
class KLDSampling:
    ABSOLUTE_MIN = 10;
    """ A helper class for KLDSampling in Python """
#/**
#   Initialize a round of KLD sampling.  Takes in kld-parameters:
#   quantile, kld-error, bin size, minimum number of samples
#**/
    def __init__(self,quantile, err, bsz, sample_min) :

        # Constructs Z-table (from ztable.data) to lookup statistics.

        self.ztable = [ 0.0000000e+00,3.9900000e-03,7.9800000e-03,1.1970000e-02,1.5950000e-02,1.9940000e-02,2.3920000e-02,2.7900000e-02,3.1880000e-02,3.5860000e-02,3.9830000e-02,4.3800000e-02,4.7760000e-02,5.1720000e-02,5.5670000e-02,5.9620000e-02,6.3560000e-02,6.7490000e-02,7.1420000e-02,7.5350000e-02,7.9260000e-02,8.3170000e-02,8.7060000e-02,9.0950000e-02,9.4830000e-02,9.8710000e-02,1.0257000e-01,1.0642000e-01,1.1026000e-01,1.1409000e-01,1.1791000e-01,1.2172000e-01,1.2552000e-01,1.2930000e-01,1.3307000e-01,1.3683000e-01,1.4058000e-01,1.4431000e-01,1.4803000e-01,1.5173000e-01,1.5542000e-01,1.5910000e-01,1.6276000e-01,1.6640000e-01,1.7003000e-01,1.7364000e-01,1.7724000e-01,1.8082000e-01,1.8439000e-01,1.8793000e-01,1.9146000e-01,1.9497000e-01,1.9847000e-01,2.0194000e-01,2.0540000e-01,2.0884000e-01,2.1226000e-01,2.1566000e-01,2.1904000e-01,2.2240000e-01,2.2575000e-01,2.2907000e-01,2.3237000e-01,2.3565000e-01,2.3891000e-01,2.4215000e-01,2.4537000e-01,2.4857000e-01,2.5175000e-01,2.5490000e-01,2.5804000e-01,2.6115000e-01,2.6424000e-01,2.6730000e-01,2.7035000e-01,2.7337000e-01,2.7637000e-01,2.7935000e-01,2.8230000e-01,2.8524000e-01,2.8814000e-01,2.9103000e-01,2.9389000e-01,2.9673000e-01,2.9955000e-01,3.0234000e-01,3.0511000e-01,3.0785000e-01,3.1057000e-01,3.1327000e-01,3.1594000e-01,3.1859000e-01,3.2121000e-01,3.2381000e-01,3.2639000e-01,3.2894000e-01,3.3147000e-01,3.3398000e-01,3.3646000e-01,3.3891000e-01,3.4134000e-01,3.4375000e-01,3.4614000e-01,3.4849000e-01,3.5083000e-01,3.5314000e-01,3.5543000e-01,3.5769000e-01,3.5993000e-01,3.6214000e-01,3.6433000e-01,3.6650000e-01,3.6864000e-01,3.7076000e-01,3.7286000e-01,3.7493000e-01,3.7698000e-01,3.7900000e-01,3.8100000e-01,3.8298000e-01,3.8493000e-01,3.8686000e-01,3.8877000e-01,3.9065000e-01,3.9251000e-01,3.9435000e-01,3.9617000e-01,3.9796000e-01,3.9973000e-01,4.0147000e-01,4.0320000e-01,4.0490000e-01,4.0658000e-01,4.0824000e-01,4.0988000e-01,4.1149000e-01,4.1308000e-01,4.1466000e-01,4.1621000e-01,4.1774000e-01,4.1924000e-01,4.2073000e-01,4.2220000e-01,4.2364000e-01,4.2507000e-01,4.2647000e-01,4.2785000e-01,4.2922000e-01,4.3056000e-01,4.3189000e-01,4.3319000e-01,4.3448000e-01,4.3574000e-01,4.3699000e-01,4.3822000e-01,4.3943000e-01,4.4062000e-01,4.4179000e-01,4.4295000e-01,4.4408000e-01,4.4520000e-01,4.4630000e-01,4.4738000e-01,4.4845000e-01,4.4950000e-01,4.5053000e-01,4.5154000e-01,4.5254000e-01,4.5352000e-01,4.5449000e-01,4.5543000e-01,4.5637000e-01,4.5728000e-01,4.5818000e-01,4.5907000e-01,4.5994000e-01,4.6080000e-01,4.6164000e-01,4.6246000e-01,4.6327000e-01,4.6407000e-01,4.6485000e-01,4.6562000e-01,4.6638000e-01,4.6712000e-01,4.6784000e-01,4.6856000e-01,4.6926000e-01,4.6995000e-01,4.7062000e-01,4.7128000e-01,4.7193000e-01,4.7257000e-01,4.7320000e-01,4.7381000e-01,4.7441000e-01,4.7500000e-01,4.7558000e-01,4.7615000e-01,4.7670000e-01,4.7725000e-01,4.7778000e-01,4.7831000e-01,4.7882000e-01,4.7932000e-01,4.7982000e-01,4.8030000e-01,4.8077000e-01,4.8124000e-01,4.8169000e-01,4.8214000e-01,4.8257000e-01,4.8300000e-01,4.8341000e-01,4.8382000e-01,4.8422000e-01,4.8461000e-01,4.8500000e-01,4.8537000e-01,4.8574000e-01,4.8610000e-01,4.8645000e-01,4.8679000e-01,4.8713000e-01,4.8745000e-01,4.8778000e-01,4.8809000e-01,4.8840000e-01,4.8870000e-01,4.8899000e-01,4.8928000e-01,4.8956000e-01,4.8983000e-01,4.9010000e-01,4.9036000e-01,4.9061000e-01,4.9086000e-01,4.9111000e-01,4.9134000e-01,4.9158000e-01,4.9180000e-01,4.9202000e-01,4.9224000e-01,4.9245000e-01,4.9266000e-01,4.9286000e-01,4.9305000e-01,4.9324000e-01,4.9343000e-01,4.9361000e-01,4.9379000e-01,4.9396000e-01,4.9413000e-01,4.9430000e-01,4.9446000e-01,4.9461000e-01,4.9477000e-01,4.9492000e-01,4.9506000e-01,4.9520000e-01,4.9534000e-01,4.9547000e-01,4.9560000e-01,4.9573000e-01,4.9585000e-01,4.9598000e-01,4.9609000e-01,4.9621000e-01,4.9632000e-01,4.9643000e-01,4.9653000e-01,4.9664000e-01,4.9674000e-01,4.9683000e-01,4.9693000e-01,4.9702000e-01,4.9711000e-01,4.9720000e-01,4.9728000e-01,4.9736000e-01,4.9744000e-01,4.9752000e-01,4.9760000e-01,4.9767000e-01,4.9774000e-01,4.9781000e-01,4.9788000e-01,4.9795000e-01,4.9801000e-01,4.9807000e-01,4.9813000e-01,4.9819000e-01,4.9825000e-01,4.9831000e-01,4.9836000e-01,4.9841000e-01,4.9846000e-01,4.9851000e-01,4.9856000e-01,4.9861000e-01,4.9865000e-01,4.9869000e-01,4.9874000e-01,4.9878000e-01,4.9882000e-01,4.9886000e-01,4.9889000e-01,4.9893000e-01,4.9896000e-01,4.9900000e-01,4.9903000e-01,4.9906000e-01,4.9910000e-01,4.9913000e-01,4.9916000e-01,4.9918000e-01,4.9921000e-01,4.9924000e-01,4.9926000e-01,4.9929000e-01,4.9931000e-01,4.9934000e-01,4.9936000e-01,4.9938000e-01,4.9940000e-01,4.9942000e-01,4.9944000e-01,4.9946000e-01,4.9948000e-01,4.9950000e-01,4.9952000e-01,4.9953000e-01,4.9955000e-01,4.9957000e-01,4.9958000e-01,4.9960000e-01,4.9961000e-01,4.9962000e-01,4.9964000e-01,4.9965000e-01,4.9966000e-01,4.9968000e-01,4.9969000e-01,4.9970000e-01,4.9971000e-01,4.9972000e-01,4.9973000e-01,4.9974000e-01,4.9975000e-01,4.9976000e-01,4.9977000e-01,4.9978000e-01,4.9978000e-01,4.9979000e-01,4.9980000e-01,4.9981000e-01,4.9981000e-01,4.9982000e-01,4.9983000e-01,4.9983000e-01,4.9984000e-01,4.9985000e-01,4.9985000e-01,4.9986000e-01,4.9986000e-01,4.9987000e-01,4.9987000e-01,4.9988000e-01,4.9988000e-01,4.9989000e-01,4.9989000e-01,4.9990000e-01,4.9990000e-01,4.9990000e-01,4.9991000e-01,4.9991000e-01,4.9992000e-01,4.9992000e-01,4.9992000e-01,4.9992000e-01,4.9993000e-01,4.9993000e-01,4.9993000e-01,4.9994000e-01,4.9994000e-01,4.9994000e-01,4.9994000e-01,4.9995000e-01,4.9995000e-01,4.9995000e-01,4.9995000e-01,4.9995000e-01,4.9996000e-01,4.9996000e-01,4.9996000e-01,4.9996000e-01,4.9996000e-01,4.9996000e-01,4.9997000e-01,4.9997000e-01,4.9997000e-01,4.9997000e-01,4.9997000e-01,4.9997000e-01,4.9997000e-01,4.9997000e-01,4.9998000e-01,4.9998000e-01,4.9998000e-01,4.9998000e-01];

        self.support_samples=0;
        self.num_samples=0;

        if (sample_min < self.ABSOLUTE_MIN):
            self.kld_samples=self.ABSOLUTE_MIN;
        else:
            self.kld_samples=sample_min;

        self.bins = [[]]; # bins is a list of lists
                          # each list is an N-dim bin of support

        confidence=quantile-0.5; # ztable is from right side of mean
        confidence=min(0.49998,max(0,confidence));

        self.max_error=err;
        self.bin_size=bsz; # list of lists

        self.zvalue=4.1;
        for i in range(len(self.ztable)):
          if (self.ztable[i] >= confidence):
            self.zvalue=i/100.0;
            break;

        print("zvalue is ",self.zvalue)


#/**
#   Update kld-sampler with the last sample drawn.  Returns a guess at
#   the number of samples needed before the distribution (which is
#   unknown) is adequately sampled.
#
#   sample = list of lists - each item is a list of dims
#**/
    def update(self,sample) :
        #print "updating with samples"
        assert len(sample[0])==len(self.bin_size), \
               "kld-sampling: Sample size not the same number of dimensions as the bins\n";

        self.num_samples = self.num_samples+1;
        if self.in_empty_bin(sample):
            self.add_to_bins(sample);
            self.support_samples = self.support_samples+1;
            if self.support_samples>=2:
                k = self.support_samples-1;
                k=math.ceil(k/(2*self.max_error)*pow(1-2/(9.0*k)+math.sqrt(2/(9.0*k))*self.zvalue,3));
                if k>self.kld_samples:
                    self.kld_samples = k;

        #print "done updating";
        return self.kld_samples;

#/**
#   Determines whether a sample falls into a bin that has already been
#   sampled.
# **/
    def in_empty_bin(self,sample):
        # init case - bins are empty
        if len(self.bins)==0:
            return True;

        # check against existing bins
        curr_bin = [ math.floor(sample[0][i]/self.bin_size[i]) for i in range(len(self.bin_size))]
        if curr_bin in self.bins:
            #print("sample", sample," is in bin ",curr_bin," already");
            return False;
        else:
            #print("sample", sample," NOT in bins ",self.bins," yet");
            return True;

        print("done.")

    def add_to_bins(self,sample):
        curr_bin = [ math.floor(sample[0][i]/self.bin_size[i]) for i in range(len(self.bin_size))]
        #print "adding sample", sample," to bins ",self.bins;
        #curr_bin = math.floor(sample/self.bin_size)
        self.bins.append(curr_bin);
        #print "done adding sample", sample," to bins ",self.bins;

In [ ]:
def rescale_img(img, scale_percent):
  width = int(img.shape[0] * scale_percent / 100)
  height = int(img.shape[1] * scale_percent / 100)
  dim = (height,width)
  # resize image
  resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
  return resized

In [ ]:
def rescale_map_coor(MAPx, MAPy, scale_percent):
  resized_mapx = []
  resized_mapy = []
  for i in range(MAPx.shape[0]):
    resized_mapx.append(int(MAPx[i] * scale_percent / 100))
    resized_mapy.append(int(MAPy[i] * scale_percent / 100))
  return resized_mapx, resized_mapy

In [ ]:
def get_new_traj(traj, skip_steps):
  new_traj = []
  for i in range(0, traj.shape[0], skip_steps):
    new_traj.append(traj[i])
  return new_traj

def get_new_obs(obs, skip_steps):
  new_obs = []
  for i in range(0, obs.shape[0], skip_steps):
    new_obs.append(obs[i])
  return new_obs

## PF-KLD

In [ ]:
def PF_KLD(Stminus1, wtminus1, observation, action, action_cov, action_alphas, map, grid_size, map_center, kld_quantile, kld_error, kld_bin_size, min_samples):
  
  num_samples=0
  S_t = []
  w_t = []
  Pf_loc = []
  # Extract motion commands
  dtrans = action[0]
  drot = action[1]
  # Map grid size
  map_px = grid_size[0]
  map_py = grid_size[1]
  # Map Center
  mapx_c = map_center[0]
  mapy_c = map_center[1]
  # Create a KLD Sampler
  sampler= KLDSampling(quantile,kld_error,bin_size,min_samples)
  # initialize low variance resampling 
  M = Stminus1.shape[0]
  ii = 0
  cc = wtminus1[0]
  rr = uniform(0, 1/M, 1)
  # KLD loop
  while (num_samples < min_samples):
    # Resample particle state from previous belief
    resampled_S_tminus1, resampled_w, ii, cc  = unit_low_variance_sampling(Stminus1, wtminus1, ii, cc, rr, num_samples)
    # Propagate the resampled particle
    current_state = propagate_particle(resampled_S_tminus1, np.array([dtrans,drot]), action_cov, action_alphas)
    S_t.append(current_state)
    num_samples = num_samples + 1
    # Compute importance weight
    # Read current Observation
    Img_test = Image.open('FC-200/' + observation).convert('LA')
    Img_test = np.array(Img_test)
    Img_test = Img_test[:,:,0]
    Img_test = rescale_img(Img_test, 33)
    #Img_test = reducing_res_img(Img_test[:,:,0], 2.75)
    Img_test = rescale_img(Img_test, LOWER_RES)
    # Crop true image from map 
    # Obtaining current pose in map coordinates
    px_gt = (current_state[0]//map_px) + mapx_c
    py_gt = (-current_state[1]//map_py) + mapy_c
    a = get_prob(Img_test, map, np.array([[py_gt,px_gt,-current_state[2]+np.pi/2]]),0.2)
    w_t.append(a)
    min_samples = sampler.update([list(current_state)])
  
  S_t = np.array(S_t)
  w_t = np.array(w_t)

  x = 0; y = 0;theta=0;
  b = w_t/np.sum(w_t)
  for i in range(min_samples):
    x = x + (b[i] * S_t[i,0])
    y = y + (b[i] * S_t[i,1])
    theta = theta + (b[i] * S_t[i,2])
  Pf_loc.append([x,y,theta])
  return Pf_loc, S_t, b, min_samples

In [ ]:
cd '/content/drive/MyDrive/Perception in Robotics project'

/content/drive/MyDrive/Perception in Robotics project


In [ ]:
# Read the dataset
df = pandas.read_csv('FC-200/metadata.csv') 

svo_x = df['SvoX'] #svo_localization
svo_y = df['SvoY']

# Derive motion commands from SVO localization 
dtrans, drot = get_svo_commands(svo_x, svo_y)

new_svox = get_new_traj(svo_x, SKIP_STEPS)
new_svoy = get_new_traj(svo_y, SKIP_STEPS)

gt_x = df['PoseX'] #gt_localization
gt_y = df['PoseY']
new_gtx = get_new_traj(gt_x, SKIP_STEPS)
new_gty = get_new_traj(gt_y, SKIP_STEPS)

mapx = df['MapX'] #gt map localization
mapy = df['MapY']
new_mapx = get_new_traj(mapx, SKIP_STEPS)
new_mapy = get_new_traj(mapy, SKIP_STEPS)

img_id = df['Filename']
new_img_id = get_new_obs(img_id, SKIP_STEPS)

# reading map 
map = Image.open('m_3808658_nw_16_1_20080717.tif').convert('LA')
map=np.array(map)
map=map[:,:,0]
map = rescale_img(map, LOWER_RES)

# We are only use map localization and gt_localization 
# to know the pixel size used because it was not mentioned
# by the author 
res_mapx, res_mapy = rescale_map_coor(np.array(new_mapx),np.array(new_mapy), LOWER_RES)
map_px = get_grid_size(res_mapx, np.array(new_gtx), cor='x')
map_py = get_grid_size(res_mapy, np.array(new_gty), cor='y')

In [ ]:
new_gtx = np.array(new_gtx)
new_gty = np.array(new_gty)
new_svox = np.array(new_svox)
new_svoy = np.array(new_svoy)

np.save('SVOX.npy', new_svox)
np.save('SVOY.npy', new_svoy)
np.save('GTX.npy', new_gtx)
np.save('GTY.npy', new_gty)

In [ ]:
Pf_Pose = []
Pf_w = []
final_loc = []

#Initialize Particle Filter 
#KLD Parameters
quantile=0.95
kld_error = 0.1
bin_size = [5,5,0.175]
min_samples=10

#Define action noise parameters
alphas = np.array([1, 1, 1])
cov = 1/3

# Define initial number of particles
num_particles = 1000

# create initial particle set
initial_state = np.array([gt_x[0],gt_y[0],0])
S_tminus1 = initiate_particles(initial_state,num_particles)

# create initial weights 
w_tminus1 = np.ones(num_particles)
w_tminus1 = w_tminus1/np.sum(w_tminus1)

Pf_w.append(w_tminus1)
Pf_Pose.append(S_tminus1)
final_loc.append([gt_x[0], gt_y[0], 0])

In [ ]:
grid = [map_px, map_py]
map_c = [res_mapx[0], res_mapy[0]]
N = []

for j in range(dtrans.shape[0]):
  print(j)
  action = [dtrans[j], drot[j]]
  obs = new_img_id[j+1]
  pose, St, wt, n = PF_KLD(Pf_Pose[-1], Pf_w[-1], obs, action, cov, alphas, map, grid, map_c, quantile, kld_error, bin_size, min_samples)
  Pf_Pose.append(St)
  Pf_w.append(wt)
  final_loc.append(pose[0])
  print(n)
  N.append(n)